In [31]:
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import meshio, numpy, copy
import meshplot as mp
import sys
import pathlib
# sys.path.append(str(pathlib.Path("/home/zizhou/smooth-ipc/build") / "debug" / "python"))  # noqa

from ipctk import *

In [32]:
V = np.array([[0.4,0],[1.0,0],[1.5,0],[1.01,0.05],[1.8,0],[0.45, -0.01]])
E = np.array([[1,0],[2,1],[4,2],[0,5],[5,4]], dtype=int)

# V = np.array([[0.5,0],[1.0,0],[1.2,0],[1.01,0.05],[1.0, -0.1]])
# E = np.array([[1,0],[2,1],[0,4],[4,2]], dtype=int)

F = np.array([])

dhat=0.1
param = ParameterType(dhat, 1, 0, 0.2, 0, 1, 200)
mesh = CollisionMesh(V, E, F)

In [33]:
# single point
def potential_single_point(points):
    cs2 = SmoothCollisions2()
    cs2.use_high_order_quadrature = False
    cs2.build(mesh, points, param)
    B = SmoothPotential(param)
    return B(cs2, mesh, points)

# IPC
def potential_IPC(points):
    cs3 = Collisions()
    cs3.build(mesh, points, param.dhat)
    B = BarrierPotential(param.dhat)
    return B(cs3, mesh, points)

In [34]:
displacements = np.linspace(-0.2, 0.2, 2000)
pA = []
pB = []
pC = []

for i, disp in enumerate(displacements):
    V_disp = copy.deepcopy(V)
    V_disp[3, 0] += disp
    
    pA.append(potential_IPC(V_disp))
    pB.append(potential_single_point(V_disp))


fig = go.Figure(data=[
    go.Scatter(x=displacements, y=pA, name="IPC"),
    go.Scatter(x=displacements, y=pB, name="Single Point"),
], layout=go.Layout(width=800, height=400))

fig.update_layout(
    yaxis_type="log"
)

fig.show()

In [35]:
xs = numpy.linspace(0.381, 0.5, 300)
ys = numpy.linspace(-0.021, 0.01, 100)
dtypes = numpy.empty((ys.size, xs.size), dtype=float)
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        V_disp = copy.deepcopy(V)
        V_disp[3, :] = [x, y]
        dtypes[j, i] = potential_single_point(V_disp)

In [38]:
fig = go.Figure(data=[
    go.Scatter(x=V[[5,0,1,2,4,5], 0], y=V[[5,0,1,2,4,5], 1],
        marker=dict(
            color='red',       # Dot color
            size=10
        ),
        line=dict(
            color='red',      # Line color
            width=2
        )),
    go.Contour(z=dtypes, x=xs, y=ys,
        contours=dict(
            start=0,   # minimum contour level
            end=200,      # maximum contour level
            size=20    # step between levels
        ))
])
fig.show()